In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.output_parsers import RegexParser
from supabase.client import create_client, Client
from dotenv import load_dotenv

In [2]:
load_dotenv("../.env")

True

In [3]:
# Initialize Supabase Client
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_API_KEY")
supabase_client: Client = create_client(url, key)

# Initialize OpenAI Client
opeani_key = url = os.environ.get('OPENAI_KEY')
client = OpenAI(api_key=opeani_key)

In [4]:
embeddings = OpenAIEmbeddings()

/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="documents_new"
    )

In [6]:
query = "Was ist Unterricht?"

In [7]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer:"""
output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser
)

In [8]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True, prompt=PROMPT)

In [9]:
query_embeddings = embeddings.embed_query(query)

2024-01-27 18:03:59,874:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [10]:
relevant_chunks = vector_store.similarity_search_by_vector_with_relevance_scores(query_embeddings,k=2)

2024-01-27 18:04:00,092:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents?limit=2 "HTTP/1.1 200 OK"


In [11]:
chunk_docs=[]
for chunk in relevant_chunks:
        chunk_docs.append(chunk[0])

In [12]:
results = chain({"input_documents": chunk_docs, "question": query})

/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
2024-01-27 18:04:01,009:INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


In [13]:
text_reference=""
for i in range(len(results["input_documents"])):
    text_reference+=results["input_documents"][i].page_content
output={"Answer":results["output_text"],"Reference":text_reference}

In [14]:
output

{'Answer': ' Unterricht is a German word that translates to "instruction" or "teaching." It refers to the process of imparting knowledge or skills to students in a formal setting, such as a classroom or online course. ',
 'Reference': 'Paulo Reglus Neves Freire (September 19, 1921 – May 2, 1997) was a Brazilian educator and\nphilosophe r who was a leading advoc ate of critical pedagogy . He is best know n for his influential work\nPedagogy  of the Oppressed, which is generally considered one of the founda tional texts of the critical\npedagogy m ovement.[26][27][28]\nConfucius (551–479 BCE) stated that authority has the respons ibility to provide oral and written instruction\nto the people unde r the rule, and "should do them good in every possible way."[21] One of the deepest\nteachings of Confucius may have been the superiority of personal exemplification over explicit rules of\nbehavior. His moral teachings emphasized self-cultivation, emulation of moral exemplars, and the\nattainme

In [15]:
def getanswer(query):
    embeddings = OpenAIEmbeddings()
    vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="documents_new",
    )
    query_embeddings = embeddings.embed_query(query)
    relevant_chunks = vector_store.similarity_search_by_vector_with_relevance_scores(query_embeddings,k=2)
    chunk_docs=[]
    for chunk in relevant_chunks:
        chunk_docs.append(chunk[0])
    results = chain({"input_documents": chunk_docs, "question": query})
    text_reference=""
    for i in range(len(results["input_documents"])):
        text_reference+=results["input_documents"][i].page_content
    output={"Answer":results["output_text"],"Reference":text_reference}
    return output

In [16]:
getanswer("Was ist Unterricht?")

2024-01-27 18:04:10,677:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-27 18:04:10,856:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents?limit=2 "HTTP/1.1 200 OK"
/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
2024-01-27 18:04:12,146:INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


{'Answer': ' Unterricht ist ein Prozess, bei dem Wissen und Fähigkeiten vermittelt werden, um das Verständnis und die Fähigkeiten der Lernenden zu fördern. Es kann in verschiedenen Formen stattfinden, wie z.B. im Klassenzimmer, online oder durch Fernunterricht.',
 'Reference': 'Paulo Reglus Neves Freire (September 19, 1921 – May 2, 1997) was a Brazilian educator and\nphilosophe r who was a leading advoc ate of critical pedagogy . He is best know n for his influential work\nPedagogy  of the Oppressed, which is generally considered one of the founda tional texts of the critical\npedagogy m ovement.[26][27][28]\nConfucius (551–479 BCE) stated that authority has the respons ibility to provide oral and written instruction\nto the people unde r the rule, and "should do them good in every possible way."[21] One of the deepest\nteachings of Confucius may have been the superiority of personal exemplification over explicit rules of\nbehavior. His moral teachings emphasized self-cultivation, emul